In [65]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.memory import ConversationBufferMemory

chat_model = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ]
)

memory = ConversationBufferMemory(
    llm=chat_model,
    max_token_limit=120,
    memory_key="movie_chat_history",
    return_messages=True,
)

In [66]:
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.schema.runnable import RunnablePassthrough

examples = [
    {
        "movie": "Inception",
        "answer": """
        🧠💭🌀
        """,
    },
    {
        "movie": "The Shawshank Redemption",
        "answer": """
        ⛓️🔨🏃
        """,
    },
    {
        "movie": "Avatar",
        "answer": """
        🌿🔵🌌
        """,
    },
    {
        "movie": "The Dark Knight",
        "answer": """
        🦇🃏🌃
        """,
    },
    {
        "movie": "Forrest Gump",
        "answer": """
        🏃🍫🎈
        """,
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Using only 3 emojis, explain about this movie: {movie}"),
        ("ai", "{answer}"),
    ]
)

few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

movie_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a movie mania and you are participating in a movie-emoji game. When introducing a movie, You have to sum up the movie in three emojis. Learn 'how to play the game' and participate in the movie-emoji game",
        ),
        ("system", "How to play the game : "),
        few_shot_prompt,
        (
            "system",
            "Let's start movie-emoji game!",
        ),
        MessagesPlaceholder(variable_name="movie_chat_history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["movie_chat_history"]

movie_chain = RunnablePassthrough.assign(movie_chat_history = load_memory) | movie_prompt | chat_model

def invoke_movie_chain(question):
    result = movie_chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content}
    )


invoke_movie_chain("please explain about 'charlie and the chocolate factory")


Sure! Here is the summary of "Charlie and the Chocolate Factory" in three emojis:

🍫🍭🎩

In [67]:
invoke_movie_chain("please explain about 'Incredibles")

Here is the summary of "Incredibles" in three emojis:

🦸‍♂️👨‍👩‍👧‍👦💥

In [68]:
invoke_movie_chain("What was the movie I just asked about?")

You just asked about "Incredibles"

In [69]:
invoke_movie_chain("What was the movie I asked before that?")

You asked about "Charlie and the Chocolate Factory"

In [70]:
invoke_movie_chain("Joker")


Here is the summary of "Joker" in three emojis:

🤡🃏🔥